In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
import os

# --- 1. Configuração do Spark e Variáveis ---
spark = SparkSession.builder \
    .appName("GoldLayerModeling") \
    .getOrCreate()

SILVER_PATH = 'hdfs://namenode:9870/datalake/silver/who_data_silver.parquet'
GOLD_PATH = 'hdfs://namenode:9870/datalake/gold/correlation_ready_data.parquet'


def load_silver_data(path):
    """Carrega os dados limpos da camada Silver do HDFS."""
    print(f"Lendo dados limpos de: {path}")
    try:
        # Lê o Parquet otimizado
        df_silver = spark.read.parquet(path)
        return df_silver
    except Exception as e:
        print(f"ERRO ao ler o Parquet em {path}: {e}")
        return spark.createDataFrame([], schema=None)


def transform_to_gold(df_silver):
    """Pivotar os dados para o modelo final (wide format)."""
    if df_silver.count() == 0:
        return spark.createDataFrame([], schema=None)

    print("Iniciando agregação e modelagem (Camada Gold)...")

    # A pivotagem no PySpark é mais limpa usando o agrupamento (groupBy)
    df_gold = df_silver.groupBy("Country_Code", "Year").pivot("Indicator_Name").agg({"Value": "mean"})

    # Renomear colunas para garantir consistência (se os nomes gerados pelo pivot forem complexos)
    # Neste caso, os nomes devem ser os mapeados na Silver:
    # 'Taxa_Mortalidade_Suicidio' e 'Prevalencia_Transtorno_Humor'

    # 1. Tratar possíveis Nulos na pivotagem (países que só têm um indicador)
    df_gold = df_gold.na.drop(subset=['Taxa_Mortalidade_Suicidio', 'Prevalencia_Transtorno_Humor'])
    
    print(f"Modelagem Gold concluída. Total de linhas prontas para correlação: {df_gold.count()}")
    return df_gold


def save_gold_data(df_gold, file_path):
    """Salva o DataFrame final no formato Parquet no HDFS."""
    if df_gold.count() > 0:
        print(f"Salvando dados agregados (Camada Gold) em: {file_path}")
        # Salvar em formato Parquet
        df_gold.write.mode("overwrite").parquet(file_path)
        print("Dados salvos com sucesso!")


if __name__ == "__main__":
    df_silver = load_silver_data(SILVER_PATH)
    if df_silver.count() > 0:
        df_gold = transform_to_gold(df_silver)
        save_gold_data(df_gold, GOLD_PATH)
    else:
        print("O processamento Gold não pode ser executado, pois a camada Silver está vazia.")
    
    spark.stop()